In [ ]:
import pandas as pd 
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import math
matplotlib.style.use('ggplot')
%matplotlib inline

In [ ]:
#Считываем данные
data = pd.read_excel('multisim_dataset.xlsx', index_col = 0)

In [ ]:
data['target'].fillna('NaN', inplace = True)
data.head(10)

In [ ]:
data.info()

In [ ]:
#Обработаем тип datetime64
data.tp_change_date = data.tp_change_date.apply(pd.to_datetime)
data['mon'] = data.tp_change_date.apply(lambda x : x.month)
data['hour'] = data.tp_change_date.apply(lambda x : x.hour)

In [ ]:
#Отделим категориальные признаки и вещественные
categorical_cols = ['sim_type', 'state_code']
numeric_cols = list(set(data.columns.values.tolist()) - set(categorical_cols + ['target'] + ['tp_change_date']))

In [ ]:
#Разделим данные для обучения на те, которые мы знаем значение target, и те, для которых нам надо его посчитать
data2 = data[data['target'] == 'NaN']
data1 = data[data['target'] != 'NaN']

In [ ]:
#Разделим на Х и у, заполним пропуски нулями для вещественных, категориальные заполним новым признаком NA
X = data1[numeric_cols + categorical_cols]
y = data1['target']

X_real_zeros = X[numeric_cols]
X_real_zeros.fillna(0, inplace = True)

X_cat = X[categorical_cols]
X_cat = pd.DataFrame(X_cat.fillna('NA'))
X.info()

In [ ]:
#Выделим уникальные категориальные признаки, закодируем их и подставим в столбец кат признаков преобразованные признаки
X_unic = pd.DataFrame(X_cat.drop_duplicates())

from sklearn.feature_extraction import DictVectorizer as DV

encoder = DV(sparse = False)
X_cat_oh = encoder.fit_transform(X_unic.T.to_dict().values())

a = []
for i in range(235):
    a.insert(i, [X_unic.values[i], X_cat_oh[i]])

X_cat_n = []
k = 0
for i in range(X_cat.shape[0]):
    for j in range(len(X_cat_oh)):
        if np.all(X_cat.values[i] == a[j][0]):
            X_cat_n.insert(k, a[j][1])
            k += 1
            break;

In [ ]:
#Разобьём на обучающую и тестовую выборку в соотношении 70/30
from sklearn.model_selection import train_test_split

(X_train_real_zeros, X_test_real_zeros, y_train, y_test) = train_test_split(X_real_zeros, y, test_size = 0.3, random_state = 0)
(X_train_cat_oh, X_test_cat_oh) = train_test_split(X_cat_n, test_size = 0.3, random_state = 0)

In [ ]:
#Соединим все признаки
X_train_z = np.hstack([X_train_real_zeros, X_train_cat_oh])
X_test_z = np.hstack([X_test_real_zeros, X_test_cat_oh])

In [ ]:
#Воспользуемся логистической регрессией с L1-регуляризацией (задача бинарной классификации)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}

y_train = y_train.astype('int')

optimizer_z = GridSearchCV(estimator = LogisticRegression(penalty='l1', solver='liblinear'), param_grid = param_grid)
optimizer_z.fit(X_train_z, y_train)

optimizer_z.best_estimator_

In [ ]:
#Посмотрим на веса при числовых признаках
optimizer_z.best_estimator_.coef_.flatten()[0:22]

In [ ]:
print(list(map(lambda x : round(x, 2), optimizer_z.best_estimator_.coef_.flatten()[0:22])))

In [ ]:
#Посчитаем метрику ROC-AUC
from sklearn.metrics import roc_auc_score
y_test = y_test.astype('int')
y_z = optimizer_z.predict(X_test_z)
auc_1 = roc_auc_score(y_test, y_z)
auc_1

### Выводы: признак hour меньше всего влияет на результат, его можно исключить из модели.

In [ ]:
#Посмотрим на данные, нужно масштабировать вещественные признаки
from pandas.plotting import scatter_matrix

data_numeric = pd.DataFrame(X_train_real_zeros, columns=numeric_cols)
list_cols = ['complex_value_size', 'tech_sms_cnt_3m', 'macro_state']
scatter_matrix(data_numeric[list_cols], alpha=0.5, figsize=(10, 10))
plt.show()

In [ ]:
#Классы сбалансированны
print(np.sum(y_train==0))
print(np.sum(y_train==1))

In [ ]:
#Уберём признак hour из модели и разобьём на обучающую и тестовую выборку
numeric_cols = list(set(data.columns.values.tolist()) - set(categorical_cols + ['target'] + ['tp_change_date'] + 
                                                            ['hour']))
X_real_zeros = X[numeric_cols]
X_real_zeros.fillna(0, inplace = True)

In [ ]:
#Масштабируем признаки и объединяем
from sklearn.preprocessing import StandardScaler

scale = StandardScaler()
X_real_scale = scale.fit_transform(X_real_zeros)

X_real_scale = np.hstack([X_real_scale, X_cat_n])

In [ ]:
#Обучаем модель
from sklearn import model_selection
y = y.astype('int').values
classifiers = []
Y = []
i = 0
skf = model_selection.StratifiedKFold(n_splits = 7, shuffle = True, random_state = 0)
for train_indices, test_indices in skf.split(X_real_scale, y):
    param_grid = {'C': [0.01, 0.05, 0.1, 0.5, 1, 5, 10]}
    x_tr = X_real_scale[train_indices]
    y_tr = y[train_indices]
    x_t = X_real_scale[test_indices]
    y_t = y[test_indices]

    optimizer_z = GridSearchCV(estimator = LogisticRegression(penalty='l2', solver='liblinear'), param_grid = param_grid)
    optimizer_z.fit(x_tr, y_tr)
    
    y_z = optimizer_z.best_estimator_.predict_proba(x_t)[:,1]
    Y.append([y_t, y_z])
    classifiers.append(optimizer_z.best_estimator_)

In [ ]:
#Считаем метрику roc-auc
auc = []
for i in range(7):
    auc.append(roc_auc_score(Y[i][0], Y[i][1]))
np.array(auc).mean()

In [ ]:
def res(c, k):
    for i in range(len(c[1])):
        if c[1][i] > k:
            c[1][i] = 1
        else:
            c[1][i] = 0
    return

In [ ]:
for i in range(len(Y)):
    res(Y[i], 0.5)

In [ ]:
v = []
from sklearn.metrics import confusion_matrix
for i in range(len(Y)):
    v.insert(i, confusion_matrix(Y[i][0], Y[i][1]))

In [ ]:
def accuracy(cnf_matrix):
    return (cnf_matrix[0][0] + cnf_matrix[1][1])/ (cnf_matrix[0][1] + cnf_matrix[1][0] + cnf_matrix[0][0] + cnf_matrix[1][1])

def recall(cnf_matrix):
    return cnf_matrix[0][0] / (cnf_matrix[0][0] + cnf_matrix[1][0])

def precision(cnf_matrix):
    return cnf_matrix[0][0] / (cnf_matrix[0][0] + cnf_matrix[0][1])

#and others :)

In [ ]:
for i in range(len(v)):
    print('accuracy: {}, recall: {}, precision: {}'.format(accuracy(v[i]), recall(v[i]), precision(v[i])))

In [ ]:
#Сделаем предсказание - усреднение ответов классификаторов
X = data2[numeric_cols + categorical_cols]

X_real_zeros = X[numeric_cols]
X_real_zeros.fillna(0, inplace = True)

X_cat = X[categorical_cols]
X_cat = pd.DataFrame(X_cat.fillna('NA'))

a = []
for i in range(235):
    a.insert(i, [X_unic.values[i], X_cat_oh[i]])

X_cat_n = []
k = 0
for i in range(X_cat.shape[0]):
    for j in range(len(X_cat_oh)):
        if np.all(X_cat.values[i] == a[j][0]):
            X_cat_n.insert(k, a[j][1])
            k += 1
            break;         
scale = StandardScaler()
X_real_scale = scale.fit_transform(X_real_zeros)

X_real_scale = np.hstack([X_real_scale, X_cat_n])
YY = []

for i in range(len(classifiers)):
    YY.append(classifiers[i].predict_proba(X_real_scale)[:,1])

for i in range(len(YY)):
    for j in range(len(YY[i])):
        if YY[i][j] > 0.5:
            YY[i][j] = 1
        else:
            YY[i][j] = 0

res = sum(YY)

for i in range(len(res)):
    if res[i] >= 4:
        res[i] = 1
    else:
        res[i] = 0
res

**Как можно улучшить данную модель**

**1) Проверка, что признаки не коррелируются между собой.**

**2) Заполнить пропуски числовых признаков среднем арифметическим по столбу.**

**3) Выбрать другой порог в логистической регрессии.**

**4) Если есть возможность, то обучиться на большем количестве данных.**

**5) Попробовать другие модели бинарной классификации (к примеру, метод опорных векторов).**